# Mortality - Scrape Actuarial Life Table

The purpose of this script is to use an API call to collect information about likelyhood of dying this year by each age.

----------------------

<p>Author: PJ Gibson</p>
<p>Date: 2022-01-19</p>
<p>Contact: peter.gibson@doh.wa.gov</p>
<p>Other Contact: pjgibson25@gmail.com</p>

## 0. Load libraries, define functions

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

Sigmoid function: 
$ y = \frac{1}{1+e^{-x}} $


Custom exponential function :
$ y = y_0 + (1 - e^{-k(x + x_0)}) $

---

Both of these functions include a multiplier in front, $z$ to help for fine-tuning

In [ ]:
def sigmoid(x, y_0, k, x_0, z):
    return z*(y_0 + ( 1 / (1+ np.e**(-k*(x+x_0)))))

def custom_exponential_function(x, y_0, k, x_0, z):
    return z*(y_0 + (1 - np.e**(-k*(x+x_0))))

## 1. Gather Data

### 1.1 Read in data

Collected from a United Nations Datasource [linked here](https://population.un.org/MarriageData/Index.html#/home).
Suggested citation: United Nations, Department of Economic and Social Affairs, Population Division (2019). World Marriage Data 2019 (POP/DB/Marr/Rev2019).
Copyright © 2019 by United Nations, made available under a Creative Commons license CC BY 3.0 IGO: http://creativecommons.org/licenses/by/3.0/igo/

*Note that the data itself for the united states 2010 data for women ever married mentions that the source of THAT data was the 2010 census.

In [ ]:
# Read in data
df_ever_married = pd.read_excel('../../../SupportingDocs/Partnerships/01_Raw/UNPD_WMD_2019_MARITAL_STATUS.xlsx', sheet_name='EVER_MARRIED', skiprows=[0,1])\
                    .rename(columns={'Country or area':'Country'})\
                    .query('(Country == "United States of America") & (YearStart == 2010) & (Sex == "Women")')

### 1.2 Minor Wrangling

In [ ]:
# The "AgeEnd" value for the 65+ group is 999. Let's make that 120 to be more realistic of the upper age a human could be
df_ever_married['AgeEnd'] = df_ever_married['AgeEnd'].replace(999,120)

# Find average (or agecenter) between start and end age values
df_ever_married['AgeCenter'] = (df_ever_married['AgeEnd'] + df_ever_married['AgeStart']) / 2

### 1.3 Inspect

Just a sanity check here to see the general shape of the curve.
This helped inform the 2 functions we defined at the top of the notebook.

In [ ]:
xs = df_ever_married['AgeCenter']
ys = df_ever_married['DataValue']

# Take a peak using matplotlib
plt.scatter(xs,ys)
plt.xlabel('Age (years)')
plt.ylabel('% Population Ever Married')
plt.show()


## 2. Line Fitting

### 2.1 Use `curve_fit()` function

In [ ]:
# Try on sigmoid (s-curve) function
pars_sigmoid, cov_sigmoid = curve_fit(f=sigmoid, xdata=xs, ydata=ys, p0=[3, 0.1, -30, 100])

# Try using decaying exponential
pars_exponential, cov2_exponential= curve_fit(f=custom_exponential_function, xdata=xs, ydata=ys, p0=[0, 0.01, -34, 90])

### 2.2 Collect fitted y-values

In [ ]:
# First define the list of x's we'll use
xs2 = np.arange(0,121)

# Use parameters and plugin x values to get y values
ys_line_sigmoid = pars_sigmoid[3]* (pars_sigmoid[0] + ( 1 / (1+ np.e**(-pars_sigmoid[1]*(xs2+pars_sigmoid[2])))))
ys_line_exponential = pars_exponential[3]* (pars_exponential[0] + (1 - np.e**(-pars_exponential[1]*(xs2+pars_exponential[2]))))

# Also calculate the average
ys_line_average = (ys_line_sigmoid + ys_line_exponential) / 2

### 2.3 Plot for visual inspection

In [ ]:
# Define our figure and it's axes
fig, ax = plt.subplots(nrows=2,ncols=2,sharex=True,sharey=True,figsize=(9,7))
ax1, ax2, ax3, ax4 = list(ax[0])+list(ax[1])

# Plot 1: Just the raw data
######################################################
ax1.scatter(xs,ys, color='black', label='raw data')


# Plot 2: Raw Data AND sigmoid (s-curve) fit.
######################################################
ax2.scatter(xs,ys, color='black')
ax2.plot(xs2,ys_line_sigmoid, color = 'red', label='sigmoid line fit')

# Plot 3: Raw Data AND decaying exponential fit.
######################################################
ax3.scatter(xs,ys, color='black')
ax3.plot(xs2,ys_line_exponential, color = 'blue', label='exponential decay line fit')

# Plot 4: For funzies average the two previous fits
######################################################
ax4.scatter(xs,ys, color='black')
ax4.plot(xs2, ys_line_average, color='purple', label='average of previous fits')

# Do a little cleanup / prettifying
for axis in [ax1,ax2,ax3,ax4]:
    axis.set_ylim(0,100)
    axis.set_xlim(-2,122)
    axis.legend(loc='lower right')

# Add title,labels
plt.suptitle('Probability of ever-being married by age: Line fits')
fig.text(0.5, 0.04, 'Age (years)', ha='center')
fig.text(0.04, 0.5, 'Probability of ever married (%)', va='center', rotation='vertical')

# Show it off!
plt.show()


### 2.4 Calculate RMSE

We'll calulate the root mean squared error to get some numerical backing for our choice in fits.

#### 2.4.1 Calculate value for few points only

For plotting, we showed all values between 0-140.
For comparing with our original data points, we only need data for a handful of points.

In [ ]:
# Use parameters and plugin x values to get y values
ys_points_sigmoid = pars_sigmoid[3]* (pars_sigmoid[0] + ( 1 / (1+ np.e**(-pars_sigmoid[1]*(xs+pars_sigmoid[2])))))
ys_points_exponential = pars_exponential[3]* (pars_exponential[0] + (1 - np.e**(-pars_exponential[1]*(xs+pars_exponential[2]))))

# Also calculate the average
ys_points_average = (ys_points_sigmoid + ys_points_exponential) / 2

#### 2.4.2 Calculate diff^2, then RMSE

We'll find the absoulte difference squared in this step.
Then calculate the mean difference squared.

In [ ]:
# Calculate Differences
diff2_sigmoid = abs(ys_points_sigmoid - ys)
diff2_exponential = abs(ys_points_exponential - ys)
diff2_average = abs(ys_points_average - ys)

# Calculate RMSE
RMSE_sigmoid = np.mean(diff2_sigmoid)
RMSE_exponential = np.mean(diff2_exponential)
RMSE_average = np.mean(diff2_average)

#### 2.4.3 Reveal Results

We see that the result with the best results (slightly) was the average of the two functions.
This agrees with my visual inspection as well.
We'll proceed with the data wrangling using this data as our choice.

In [ ]:
print(f'{np.round(RMSE_sigmoid,4)}: RMSE of the sigmoid function approximation')
print(f'{np.round(RMSE_exponential,4)}: RMSE of our custom exponential function approximation')
print(f'{np.round(RMSE_average,4)}: RMSE of the average values between two function outputs approximation')

## 3. Wrangle Data

Now that we have a good idea of what percent of the population (female) is ever_married for each individual age (per our average line-fit), we want to dive into our primary question:

<b>What is the likelyhood of getting married for the first time at any given age?</b>

----

We'll tackle this problem using a manual approach wher and see how many individuals are married off each year.
In our manual compartementalized approach we'll use the following logic:
* Start with 100 unmarried 18-year-olds tracking their marriage status each year
  * note that while we say 100 unmarried 18-year-olds, we'll still have fractions and don't ever round.
* Each year, by the end of the year, the total number ever_married must equal the percert given in our linear approximation for that age.
* The probability of getting married in a given year is the number married this year / number never married at the beginning of the year.

*this number of 100 is somewhat arbirary but helps easily see the relationship between the fields "PercentMarried" and "NumMarried_ThisYear".

### 3.1 Establish Original Dataframe

Only contains the age and perent of individuals (of that age) that are married.

The simulation will force partnerships to occur at 18 years of age or later.
This isn't perfectly represntative of the United States since some partnerships/marriages do happen before that age, but I don't want a simulation that includes that type of partnership.

In [ ]:
# Get percent of population married at each age using our 2 line average fit.
df = pd.DataFrame(data = np.column_stack([xs2,ys_line_average]),\
                  columns = ['Age','PercentMarried'])

# Only look at ages older than 17.  Initialize so that 0 17-year-olds are married. PJ's choice.
df = df.query('Age >= 17')
df.loc[17,'PercentMarried'] = 0

### 3.2 Establish Probability Related Fields

In [ ]:
# Given an initial population of 100, find the number single
df['NumSingle'] = 100 - df['PercentMarried']

# Grab the population of last year (only considering singles)
df['NumSingle_LastYear'] = np.append(np.nan,np.array(df['NumSingle'][:-1]))

# Find out how many people were married this year
df['NumMarried_ThisYear'] = df['NumSingle_LastYear'] - df['NumSingle']

# Probability of first marriage this year = number married / number initially single
df['ProbabilityOfMarriage_ThisYear'] = df['NumMarried_ThisYear'] / df['NumSingle_LastYear']

## 4. Saving

### 4.1 Save to .csv

Note we don't use this .csv in the process, but it is interesting to have a copy of.

In [ ]:
df.query('Age > 17')\
  .to_csv('../../../SupportingDocs/Partnerships/03_Complete/ProbabilityMarriageThisYear_byAge.csv', index=False)

### 4.2 Print output

This is what is copy/pasted into the notebook "{root}/Functions/Process Functions" in the `spdf_will_marry()` function in the 2. Partnership Process section of the notebook.
Note that we define the list instead of reading in a .csv file each time due to the nature of UDFs.
We don't want to reread the same .csv file over and over again for every row, it's more timely just to define the list within the function.

In [ ]:
df.query('Age > 17')[['Age','ProbabilityOfMarriage_ThisYear']].to_numpy()

## 5. Extra Sanity Checks

### 5.1 See if our probabilities check out

In [ ]:

# Calculate the inverse probability -> probability of staying single that year
df['inverseProba'] = 1- df['ProbabilityOfMarriage_ThisYear']

# For each row (not the first for 17 year olds):
for i in np.arange(1,len(df)):

    # Find the probabilities up to that point
    subset = df['inverseProba'].iloc[1:i+1]

    # Find current row age
    cur_age = df.iloc[i]['Age']

    # Calculate product of all previous probabilies of staying single
    starter = 100
    for element in subset:
        starter = starter*element

    # Calc the difference between number single and product of percent chances of leaving every previous year single.
    ### Should be the same
    difference = df.iloc[i].NumSingle - starter

    # If they're not the same, call me a monkey's uncle.
    if difference > 0.000000001:
        print(f'YOU FOOL!!!!\n At {cur_age}, difference was {difference} which is way too much!')



### 5.2 Fun Plotting

In [ ]:
age_highest_proba = df.iloc[df['ProbabilityOfMarriage_ThisYear'].argmax()]['Age']

plt.plot(df['Age'], df['ProbabilityOfMarriage_ThisYear'], label='probability')
plt.vlines(x=age_highest_proba, ymin=0,ymax=0.1, colors='k',linestyles='--',label=f'max likelyhood if unmarried:\nage {int(age_highest_proba)}')
plt.xlabel('Age')
plt.ylabel('Probability (normalized to 1.0)')
plt.title('Probability of Marriage by Age (if unmarried by that age)')
plt.legend(loc='upper right')
plt.xlim(18,120)
plt.ylim(0,0.1)
plt.show()

In [ ]:
plt.plot(df['Age'],df['PercentMarried'])
plt.xlim(18,120)
plt.ylim(0,100)
plt.xlabel('Age')
plt.ylabel('Percent of Population Ever Married')
plt.title('Percent of Population Married by Age')
plt.show()